<a href="https://colab.research.google.com/github/rishipython/ALL-Cell-Classification/blob/main/LeukemiaClassificationTraining1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import ResNet50
from keras.layers import Add
from keras.layers import Dense
from keras import backend as K
from keras.engine import Input, Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Activation, BatchNormalization, PReLU, Dropout, Conv2DTranspose
from keras.optimizers import Adam
from keras.layers.merge import concatenate
from keras.utils import to_categorical
import PIL
import numpy as np
from PIL import Image
from numpy import asarray
import glob
import nibabel as nib
import os
import tarfile as trf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import roc_auc_score, roc_curve, auc
from IPython.display import display
K.set_image_data_format("channels_last")

In [ ]:
# Access GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Apr 11 15:56:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Access more memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# See if Tensorflow is using GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_dir = f"{os.sep}content{os.sep}drive{os.sep}My Drive{os.sep}leukemia_classification{os.sep}"

# Make and train model_1, model_2, and model_3

In [ ]:
x_train = np.load(drive_dir+"x_train.npy")
y_train = np.load(drive_dir+"y_train.npy")
x_val = np.load(drive_dir+"x_val.npy")
y_val = np.load(drive_dir+"y_val.npy")
x_test = np.load(drive_dir+"x_test.npy")
y_test = np.load(drive_dir+"y_test.npy")

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1020, 450, 450, 3)
(1020, 1, 1, 1)
(340, 450, 450, 3)
(340, 1, 1, 1)
(340, 450, 450, 3)
(340, 1, 1, 1)


In [ ]:
mean = float(open(drive_dir+"mean.txt", 'r').readline())
std = float(open(drive_dir+"std.txt", 'r').readline())

In [ ]:
print(mean)
print(type(mean))
print(std)
print(type(std))

9.178323056564189
<class 'float'>
23.645296284294893
<class 'float'>


In [ ]:
input_shape = x_train[0].shape
output_shape = (1, 1, 1)

In [ ]:
print(input_shape)
print(output_shape)

(450, 450, 3)
(1, 1, 1)


In [ ]:
lr = 1e-4
loss_function = "binary_crossentropy"
metrics = "accuracy"
if not isinstance(metrics, list):
  metrics = [metrics]

In [ ]:
# Model 1 (VGG16)
model_1_input_layer = Input(shape=input_shape)
model_1_vgg16 = VGG16(input_shape=input_shape, include_top=False)(model_1_input_layer)
model_1_output_layer = Conv2D(output_shape[-1], model_1_vgg16.shape[1]-output_shape[0]+1, activation="sigmoid")(model_1_vgg16)
model_1 = Model(model_1_input_layer, model_1_output_layer)
model_1.compile(optimizer = Adam(lr = lr), loss = loss_function, metrics = metrics)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
model_1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 450, 450, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, 14, 14, 512)       14714688  
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 1, 1)           100353    
Total params: 14,815,041
Trainable params: 14,815,041
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Model 2 (ResNet50)
model_2_input_layer = Input(shape=input_shape)
model_2_resnet50 = ResNet50(input_shape=input_shape, include_top=False)(model_2_input_layer)
model_2_output_layer = Conv2D(output_shape[-1], model_2_resnet50.shape[1]-output_shape[0]+1, activation="sigmoid")(model_2_resnet50)
model_2 = Model(model_2_input_layer, model_2_output_layer)
model_2.compile(optimizer = Adam(lr = lr), loss = loss_function, metrics = metrics)

94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
model_2.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 450, 450, 3)]     0         
_________________________________________________________________
resnet50 (Functional)        (None, 15, 15, 2048)      23587712  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 1, 1, 1)           460801    
Total params: 24,048,513
Trainable params: 23,995,393
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
# Model 3 (InceptionV3)
model_3_input_layer = Input(shape=input_shape)
model_3_inception_v3 = InceptionV3(input_shape=input_shape, include_top=False)(model_3_input_layer)
model_3_output_layer = Conv2D(output_shape[-1], model_3_inception_v3.shape[1]-output_shape[0]+1, activation="sigmoid")(model_3_inception_v3)
model_3 = Model(model_3_input_layer, model_3_output_layer)
model_3.compile(optimizer = Adam(lr = lr), loss = loss_function, metrics = metrics)

87916544/87910968 [==============================] - 3s 0us/step


In [ ]:
model_3.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 450, 450, 3)]     0         
_________________________________________________________________
inception_v3 (Functional)    (None, 12, 12, 2048)      21802784  
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 1, 1, 1)           294913    
Total params: 22,097,697
Trainable params: 22,063,265
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
models = [model_1, model_2, model_3]

In [ ]:
histories = [None for i in range(len(models))]

In [ ]:
epochs = 33
batch_size = 10

In [ ]:
# Train models
for i in range(len(models)):
  histories[i] = models[i].fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)
  models[i].save(drive_dir + f'model_{i+1}')
  hist_df = pd.DataFrame(histories[i].history)
  hist_csv_file = drive_dir + f'history_{i+1}.csv'
  with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
  print(f"Completed training of model_{i+1}.")

Epoch 1/33
102/102 [==============================] - 58s 358ms/step - loss: 0.5938 - accuracy: 0.6799 - val_loss: 0.5302 - val_accuracy: 0.7706
Epoch 2/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4798 - accuracy: 0.7822 - val_loss: 0.5110 - val_accuracy: 0.7471
Epoch 3/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4892 - accuracy: 0.7678 - val_loss: 0.5407 - val_accuracy: 0.7794
Epoch 4/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4960 - accuracy: 0.7731 - val_loss: 0.4573 - val_accuracy: 0.8000
Epoch 5/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4486 - accuracy: 0.8029 - val_loss: 0.4931 - val_accuracy: 0.7765
Epoch 6/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4548 - accuracy: 0.7989 - val_loss: 0.5418 - val_accuracy: 0.7176
Epoch 7/33
102/102 [==============================] - 35s 343ms/step - loss: 0.4317 - accuracy: 0.8182 - val_loss: 0.4448 - val_ac